## GoogleDriveをマウントする

In [38]:
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
base_path = '/content/drive/My Drive/Images'
# os.listdir(base_path)

## ライブラリをインポート

In [0]:
import tensorflow as tf
import cv2
import random, json
import numpy as np
import pandas as pd

from tensorflow.contrib.tpu.python.tpu import keras_support
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import regularizers
from tensorflow.keras import optimizers, utils
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ProgbarLogger, ReduceLROnPlateau, LambdaCallback
from tensorflow.keras.layers import Input

from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## データロード用のクラスを定義

In [0]:
class DataSequence(Sequence):
    def __init__(self, data_path, label, batch_size, is_valid=False):
        self.batch = batch_size
        self.data_file_path = data_path
        self.datagen = ImageDataGenerator(
                            rotation_range=15,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            # zoom_range=0.2,
                            # horizontal_flip=True,
                            # channel_shift_range=3.,
                            brightness_range=[0.7, 1.3]
                        )
        self.is_valid = is_valid
        d_list = os.listdir(self.data_file_path)
        self.f_list = []
        for dir in d_list:
            if dir == 'empty': continue
            for f in os.listdir(self.data_file_path+'/'+dir):
                self.f_list.append(self.data_file_path+'/'+dir+'/'+f)
        self.label = label
        self.length = len(self.f_list)

    def __getitem__(self, idx):
        warp = self.batch
        aug_time = 3 if not self.is_valid else 0
        datas, labels = [], []
        label_dict = self.label

        for f in random.sample(self.f_list, warp):
            img = cv2.imread(f)
            img = cv2.resize(img, (160, 160))
            img = img.astype(np.float32) / 255.0
            datas.append(img)
            label = f.split('/')[-2].split('_')[-1]
            labels.append(label_dict[label])
            # Augmentation image
            for num in range(aug_time):
                tmp = self.datagen.random_transform(img)
                datas.append(tmp)
                labels.append(label_dict[label])

        datas = np.asarray(datas)
        labels = pd.DataFrame(labels)
        labels = utils.to_categorical(labels, len(label_dict))
        return datas, labels

    def __len__(self):
        return self.length

    def on_epoch_end(self):
        ''' 何もしない'''
        pass

## モデル転移用のクラスを定義

In [0]:
class CustumModel():
    def __init__(self):
        '''
        学習済みモデルのロード(base_model)
        '''
        model_path = '/content/drive/My Drive/models/facenet_keras.h5'
        self.base_model = load_model(model_path)

    def createModel(self, label_dict):
        '''
        転移学習用のレイヤーを追加
        '''
        added_layer = GlobalAveragePooling2D()(self.base_model.layers[-5].output)
        added_layer = Dense(512, kernel_regularizer=regularizers.l2(0.01))(added_layer)
        added_layer = BatchNormalization()(added_layer)
        added_layer = Activation('relu')(added_layer)
        added_layer = Dense(len(label_dict), activation='softmax', name='classification')(added_layer)

        '''
        base_modelと転移学習用レイヤーを結合
        '''
        model = Model(inputs=self.base_model.input, outputs=added_layer)

        '''
        base_modelのモデルパラメタは学習させない。
        (added_layerのモデルパラメタだけを学習させる)
        '''
        for layer in self.base_model.layers:
            layer.trainable = False
        model.summary()

        return model

##   学習用画像のロード

In [43]:
label_dict = {}
count = 0
batch_size = 64
for d_name in os.listdir(base_path):
    if d_name == 'empty': continue
    if d_name == '.DS_Store': continue
    d_name = d_name.split('_')[-1]
    label_dict[d_name] = count
    count += 1
print(label_dict)
train_gen = DataSequence(base_path, label_dict, batch_size)
validate_gen = DataSequence(base_path, label_dict, batch_size, True)

{'SY': 0, 'RN': 1, 'HU': 2, 'YY': 3, 'RM': 4, 'TO': 5, 'SM': 6, 'SS': 7}


## カスタムモデルオブジェクトを生成

In [44]:
cm = CustumModel()
model = cm.createModel(label_dict)
callbacks = [
    ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=3, min_lr=1e-10, verbose=1),
    # ModelCheckpoint('./checkpoint/weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1, mode='auto')
]
tf.test.gpu_device_name()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
____________________________________________________________________________________________

'/device:GPU:0'

In [45]:
'''
全体のモデルをコンパイル
'''
# opt = optimizers.Adam(lr=1e-4)
opt = optimizers.RMSprop(lr=1e-3)
# opt = optimizers.SGD(lr=1e-3)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(
    train_gen,
    epochs=50,
    steps_per_epoch=int(train_gen.length / batch_size),
    callbacks=callbacks,
    validation_data=validate_gen,
    validation_steps=int(validate_gen.length / batch_size),
)

model.save('/content/drive/My Drive/models/polaris_facenet_model.h5')
l_dict = {label_dict[name]: name for name in label_dict.keys()}
json.dump(l_dict, open('/content/drive/My Drive/models/polaris_labels.json', 'w'), indent=4)

Epoch 1/50
3/3 [==============================] - 19s 6s/step - loss: 9.9081 - acc: 0.3815 - val_loss: 8.4612 - val_acc: 0.7344
Epoch 2/50
3/3 [==============================] - 3s 1s/step - loss: 8.2375 - acc: 0.6198 - val_loss: 7.7589 - val_acc: 0.7656
Epoch 3/50
3/3 [==============================] - 3s 1s/step - loss: 7.3482 - acc: 0.6914 - val_loss: 7.1160 - val_acc: 0.8073
Epoch 4/50
3/3 [==============================] - 3s 1s/step - loss: 6.7775 - acc: 0.7044 - val_loss: 6.5598 - val_acc: 0.8125
Epoch 5/50
3/3 [==============================] - 3s 1s/step - loss: 6.1943 - acc: 0.7057 - val_loss: 6.0974 - val_acc: 0.7604
Epoch 6/50
3/3 [==============================] - 3s 999ms/step - loss: 5.6760 - acc: 0.7201 - val_loss: 5.5926 - val_acc: 0.7865
Epoch 7/50
3/3 [==============================] - 3s 1s/step - loss: 5.2169 - acc: 0.7227 - val_loss: 5.1654 - val_acc: 0.8281
Epoch 8/50
3/3 [==============================] - 3s 1s/step - loss: 4.7555 - acc: 0.7344 - val_loss: 4.739

In [0]:
# model = load_model('/content/drive/My Drive/models/polaris_custumvgg19_model.h5', compile=False)
# callbacks = [
#     ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-10, verbose=1),
#     # ModelCheckpoint('./checkpoint/weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1, mode='auto')
# ]


# # opt = optimizers.Adam(lr=1e-4)
# # opt = optimizers.RMSprop(lr=1e-4)
# opt = optimizers.SGD(lr=1e-5)
# model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# model.fit_generator(
#     train_gen,
#     epochs=5,
#     steps_per_epoch=int(train_gen.length / batch_size),
#     callbacks=callbacks,
#     validation_data=validate_gen,
#     validation_steps=int(validate_gen.length / batch_size),
# )
# model.save('/content/drive/My Drive/models/polaris_custumxception_model.h5')